# This notebook is to test a single batch run in ADAM

In [ ]:
from adam import Batch
from adam import Batches
from adam import BatchRunManager
from adam import PropagationParams
from adam import OpmParams
from adam import ConfigManager
from adam import Projects
from adam import RestRequests
from adam import AuthenticatingRestProxy
import time
import os

This sets up authenticated access to the server. It needs to be done before pretty much everything you want to do with ADAM.

In [2]:
# Reads your config from a file in current directory. For instructions on setting this up, see config_demo notebook.
config = ConfigManager().get_config(environment='dev')
auth_rest = AuthenticatingRestProxy(RestRequests(config['url']), config['token'])

## Example Inputs

In [3]:
# 6x1 state vector (position [km], velocity [km/s])
# asteroid 101
state_vec3 = [-150874809.2,
              -187234595.3,
              -73785026.7,
              14.64403935,
              -11.75744819,
              -5.583528281]

### Set Parameters

Commented parameters are optional. Uncomment to use.

In [4]:
propagation_params3 = PropagationParams({
    'start_time': '2001-01-02T01:13:46.620000Z',   # propagation start time in ISO format
    'end_time': '2010-12-01T01:13:46.620000Z',     # propagation end time in ISO format

    'project_uuid': config['workspace'],

     'step_size': 86400,                   # step size (seconds)
     'propagator_uuid': '00000000-0000-0000-0000-000000000001',  # force model: sun, moon, planets
#     'description': 'some description'       # description of run
})

opm_params3 = OpmParams({
    'epoch': '2001-01-02T01:13:46.620000Z',
    'state_vector': state_vec3,
})

### Submit and Run Propagation

In [5]:
batch3 = Batch(propagation_params3, opm_params3)
print("Submitting OPM:")
print(batch3.get_opm_params().generate_opm())

Submitting OPM:
CCSDS_OPM_VERS = 2.0
CREATION_DATE = 2020-05-04 23:30:33.615480
ORIGINATOR = ADAM_User
COMMENT Cartesian coordinate system
OBJECT_NAME = dummy
OBJECT_ID = 001
CENTER_NAME = SUN
REF_FRAME = ICRF
TIME_SYSTEM = UTC
EPOCH = 2001-01-02T01:13:46.620000Z
X = -150874809.2
Y = -187234595.3
Z = -73785026.7
X_DOT = 14.64403935
Y_DOT = -11.75744819
Z_DOT = -5.583528281
MASS = 1000.0
SOLAR_RAD_AREA = 20.0
SOLAR_RAD_COEFF = 1.0
DRAG_AREA = 20.0
DRAG_COEFF = 2.2



In [6]:
# Submit and wait until batch run is ready
batches_module = Batches(auth_rest)
BatchRunManager(batches_module, [batch3]).run()

[18.008176] Submitting 1 runs.
[17.100487] Running.
[0.917767] Retrieving propagation results.


### Get Status and Parts Count

In [7]:
# Get final status and parts count
parts_count = batch3.get_state_summary().get_parts_count()
print("Final state: %s, part count %s\n" % (batch3.get_calc_state(), parts_count))

Final state: COMPLETED, part count 1



### Get Ephemeris of Specified Part

In [8]:
# Get ephemeris of specified part
part_to_get = 0
eph3 = batch3.get_results().get_parts()[part_to_get].get_ephemeris()
print("Ephemeris:")
print(eph3)

Ephemeris:
stk.v.11.0
BEGIN Ephemeris
ScenarioEpoch 02 Jan 2001 01:13:46.620000
CentralBody SUN
CoordinateSystem ICRF
InterpolationMethod HERMITE
InterpolationOrder 5
NumberOfEphemerisPoints 3622

EphemerisTimePosVel
0.000000000000e+00 -1.508748092000e+11 -1.872345953000e+11 -7.378502670000e+10 1.464403935000e+04 -1.175744819000e+04 -5.583528281000e+03
8.640000000000e+04 -1.496048834148e+11 -1.882446034321e+11 -7.426514310215e+10 1.475220911341e+04 -1.162228518255e+04 -5.530234181390e+03
1.728000000000e+05 -1.483256588396e+11 -1.892429116249e+11 -7.474064373278e+10 1.485928948624e+04 -1.148661680272e+04 -5.476681963632e+03
2.592000000000e+05 -1.470372295041e+11 -1.902294763535e+11 -7.521150635354e+10 1.496528261632e+04 -1.135044626344e+04 -5.422873080551e+03
3.456000000000e+05 -1.457396892566e+11 -1.912042543673e+11 -7.567770884990e+10 1.507019056415e+04 -1.121377669472e+04 -5.368808946233e+03
4.320000000000e+05 -1.444331317710e+11 -1.921672026822e+11 -7.613922922788e+10 1.517401530257

### Get ending state vector

In [9]:
# Get the end state vector (uncomment to use)
end_state_vector3 = batch3.get_results().get_end_state_vector()
print("State vector at the end of propagation:")
print(end_state_vector3)
print("should be: [62739872.06, 117402282.4, 47549221.51, -31.06961002, 13.64009464, 7.070881807]")

State vector at the end of propagation:
[62740925.88694, 117401828.3091, 47548985.9621, -31.069513066960003, 13.64027732597, 7.0709550437290005]
should be: [62739872.06, 117402282.4, 47549221.51, -31.06961002, 13.64009464, 7.070881807]
